In [2]:
import os
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import gradio as gr

In [3]:
def load_emnist_data(base_path):
    data = []
    labels = []

    train_path = os.path.join(base_path, 'Train')
    validation_path = os.path.join(base_path, 'Validation')

    def load_data_from_folder(folder_path, label):
        images = []
        for filename in os.listdir(folder_path):
            if filename.endswith(('.jpg', '.png')):
                img_path = os.path.join(folder_path, filename)
                img = io.imread(img_path, as_gray=True)  
                img = transform.resize(img, (28, 28))  
                images.append(img)
                labels.append(label)
        return images

    for label in os.listdir(train_path):
        label_path = os.path.join(train_path, label)
        if os.path.isdir(label_path):
            label_value = ord(label) 
            data += load_data_from_folder(label_path, label_value)

    for label in os.listdir(validation_path):
        label_path = os.path.join(validation_path, label)
        if os.path.isdir(label_path):
            label_value = ord(label)  
            data += load_data_from_folder(label_path, label_value)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

In [4]:
base_path = 'C:/Users/Adam/Desktop/WTUM/dataset/'

In [5]:
data, labels = load_emnist_data(base_path)

In [6]:
datan = data / 255.0

In [7]:
X_train, X_test, y_train, y_test = train_test_split(datan, labels, test_size=0.2, random_state=42)

In [8]:
unique_labels = np.unique(labels)
num_classes = len(unique_labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
y_train_mapped = np.array([label_to_index[label] for label in y_train])
y_test_mapped = np.array([label_to_index[label] for label in y_test])

y_train = to_categorical(y_train_mapped, num_classes)
y_test = to_categorical(y_test_mapped, num_classes)

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(256, activation='relu'),
    Dropout(0.5),  
    Dense(128, activation='relu'),
    Dropout(0.5),  
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),  
              loss='categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\Adam\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 24s 996us/step - accuracy: 0.5144 - loss: 1.7050 - val_accuracy: 0.8095 - val_loss: 0.6292
Epoch 2/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 973us/step - accuracy: 0.7312 - loss: 0.9049 - val_accuracy: 0.8336 - val_loss: 0.5328
Epoch 3/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 971us/step - accuracy: 0.7542 - loss: 0.8185 - val_accuracy: 0.8474 - val_loss: 0.4812
Epoch 4/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 967us/step - accuracy: 0.7662 - loss: 0.7753 - val_accuracy: 0.8550 - val_loss: 0.4581
Epoch 5/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 971us/step - accuracy: 0.7726 - loss: 0.7516 - val_accuracy: 0.8591 - val_loss: 0.4467
Epoch 6/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 971us/step - accuracy: 0.7789 - loss: 0.7301 - val_accuracy: 0.8645 - val_loss: 0.4306
Epoch 7/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 969us/step - accuracy: 0.7821 - loss: 0.7192 - val_accuracy: 0.8651 - val_loss: 0.4308
Epoch 8/10
21414/21414 ━━━━━━━━━━━━━━━━━━━━ 21s 969us/s

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

5354/5354 ━━━━━━━━━━━━━━━━━━━━ 3s 488us/step - accuracy: 0.8735 - loss: 0.3999
Loss: 0.403949499130249, Accuracy: 0.8726767301559448


In [26]:
model.save('C:/Users/Adam/Desktop/WTUM/snn_model_2.h5')